In [22]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [23]:
files = glob.glob("/content/drive/MyDrive/train/train/*.csv")
data_list = []
for file in files:
  data_list.append(pd.read_csv(file, index_col=0))
df = pd.concat(data_list)  

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [24]:
def data_pre(df):
  null_list = []
  for col in df.columns:
    count = df[col].count()
    if count == 0:
      null_list.append(col)
  df = df.drop(null_list, axis=1)

  df = df.drop("市区町村名", axis=1)

  df = df.drop("種類", axis=1)

  dis = {
      "30分?60分":45,
      "1H?1H30":75,
      "2H?":120,
      "1H30?2H":105
  }
  df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)

  df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上", 2000).astype(float)

  y_list = {}
  for i in df["建築年"].value_counts().keys():
    if "平成" in i:
      num = float(i.split("平成")[1].split("年")[0])
      year = 33 - num
    if "令和" in i:
      num = float(i.split("令和")[1].split("年")[0])
      year = 3 - num
    if "昭和" in i:
      num = float(i.split("昭和")[1].split("年")[0])
      year = 96 - num
    y_list[i] = year
  y_list["戦前"] = 76.0
  df["建築年"] = df["建築年"].replace(y_list)

  year = {
      "年第１四半期": ".25",
      "年第２四半期": ".50",
      "年第３四半期": ".75",
      "年第４四半期": ".99"
  }
  year_list = {}
  for i in df["取引時点"].value_counts().keys():
    for k, j in year.items():
      if k in i:
        year_rep = i.replace(k, j)
    year_list[i] = year_rep
  df["取引時点"] = df["取引時点"].replace(year_list).astype(float)

  for col in ["都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]:
    df[col] = df[col].astype("category")

  return df

In [25]:
df = data_pre(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 637351 entries, 19001200 to 39012211
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   市区町村コード       637351 non-null  int64   
 1   都道府県名         637351 non-null  category
 2   地区名           637060 non-null  category
 3   最寄駅：名称        634732 non-null  category
 4   最寄駅：距離（分）     614306 non-null  float64 
 5   間取り           615609 non-null  category
 6   面積（㎡）         637351 non-null  float64 
 7   建築年           619117 non-null  float64 
 8   建物の構造         623423 non-null  category
 9   用途            591214 non-null  category
 10  今後の利用目的       275091 non-null  category
 11  都市計画          618786 non-null  category
 12  建ぺい率（％）       614848 non-null  float64 
 13  容積率（％）        614848 non-null  float64 
 14  取引時点          637351 non-null  float64 
 15  改装            580431 non-null  category
 16  取引の事情等        18275 non-null   category
 17  取引価格（総額）_log  637351

In [26]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

df_train, df_val = train_test_split(df, test_size=0.2)

col = "取引価格（総額）_log"
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

params = {
    "objective": "regression",
    "metrics": "mae"
}

model = lgb.train(params, trains, valid_sets=valids, num_boost_round=1000, early_stopping_rounds=100)

/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	valid_0's l1: 0.253701
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's l1: 0.237246
[3]	valid_0's l1: 0.222798
[4]	valid_0's l1: 0.209762
[5]	valid_0's l1: 0.198442
[6]	valid_0's l1: 0.188437
[7]	valid_0's l1: 0.179697
[8]	valid_0's l1: 0.171725
[9]	valid_0's l1: 0.164722
[10]	valid_0's l1: 0.158246
[11]	valid_0's l1: 0.1528
[12]	valid_0's l1: 0.147723
[13]	valid_0's l1: 0.143288
[14]	valid_0's l1: 0.139401
[15]	valid_0's l1: 0.135843
[16]	valid_0's l1: 0.132825
[17]	valid_0's l1: 0.129835
[18]	valid_0's l1: 0.127338
[19]	valid_0's l1: 0.124695
[20]	valid_0's l1: 0.122329
[21]	valid_0's l1: 0.120346
[22]	valid_0's l1: 0.118416
[23]	valid_0's l1: 0.116606
[24]	valid_0's l1: 0.114908
[25]	valid_0's l1: 0.113288
[26]	valid_0's l1: 0.111956
[27]	valid_0's l1: 0.110711
[28]	valid_0's l1: 0.109386
[29]	valid_0's l1: 0.108182
[30]	valid_0's l1: 0.107208
[31]	valid_0's l1: 0.106291
[32]	valid_0's l1: 0.105319
[33]	valid_0's l1: 0.104454
[34]	valid_0's l1: 0.103

In [27]:
vals = model.predict(val_x)
mae(vals, val_y)

0.07678341846991599

In [28]:
df_test = pd.read_csv("/content/drive/MyDrive/test.csv", index_col=0)
df_test = data_pre(df_test)

In [30]:
predict = model.predict(df_test)
df_test["取引価格（総額）_log"] = predict
df_test[["取引価格（総額）_log"]].to_csv("submit_test.csv")

In [31]:
pd.DataFrame(model.feature_importance(), index=val_x.columns, columns=["importance"]).sort_values("importance", ascending = False)

,importance
地区名,8509
最寄駅：名称,6371
建築年,3409
取引時点,3127
面積（㎡）,2120
最寄駅：距離（分）,2041
間取り,853
市区町村コード,735
都道府県名,558
改装,512
